## Processing neural datasets
Process neural datasets into python structures for smaller storage size and faster access

In [1]:
%load_ext autoreload
%autoreload 2


import neural_datasets

import os
if not os.path.exists('../data'):
    os.makedirs('../data')
if not os.path.exists('../data/datasets'):
    os.makedirs('../data/datasets')


datadir = '/scratches/ramanujan_2/dl543/'
datadir2 = '/scratches/sagarmatha_2/ktj21/data/crcns/'

We load the dataset for one session from the original files downloaded from [CRCNR.org](https://crcns.org/), which is then preprocessed and put into a pickle file in the ```../data/datasets``` directory.

In [5]:
# Mice sleep/wake exploration, head direction cells
datadir2 = '/scratches/sagarmatha_2/ktj21/data/crcns/th-1/data/'
k = 2

if k == 0:
    session_id = 'Mouse12-120806'
    mouse_id = 'Mouse12'
    channels = {
        'ANT': [1, 2, 3, 4, 5, 6, 7, 8],
        'mPFC': [9, 10, 11, 12],
        'CA1': [13]
                   }
elif k == 1:
    session_id = 'Mouse28-140313'
    mouse_id = 'Mouse28'
    channels = {
        'PoS': [1, 2, 3, 4, 5, 6, 7], 
        'ANT': [8, 9, 10, 11]
            }
elif k == 2:
    session_id = 'Mouse24-131213'
    mouse_id = 'Mouse24'
    channels = {
        #'PoS': [1, 2, 3, 4],
        'ANT': [5, 6, 7, 8]
    }   
else:
    raise ValueError

# behav_tbin = 0.025599999999940337

phase = 'wake'
savef = '../data/datasets/th1_{}_{}.p'.format(session_id, phase)
neural_datasets.peyrache_th1(datadir2, mouse_id, session_id, savef, channels, phase)

Empty channel shank 0 cluster 1.
Behaviour time bin size 0.0256 s.
